**Section 221**

Data cleaning prep:
1. find any na values
2. Standardize column names:
3. remove redundant columns (County_Name & county_town_name) / other formatting issues
4. ensure column names are represented correctly as objects (categorical vs numerical)
5. add geographic identifier
6. aggregate standardized data by county
7. normalized income eligibility threshold

In [ ]:
library(dplyr)
library(readr)
library(stringr)
library(scales)

df <- read_csv("/content/section221.csv")
colnames(df) <- tolower(gsub(" ", "_", colnames(df)))

#3
df <- df %>%
  mutate(
    county_name = str_to_title(str_trim(county_name)),
    county_town_name = str_to_title(str_trim(county_town_name))
  )

#4
categorical_cols <- c("state_alpha", "state_name", "hud_area_code", "hud_area_name", "county_name", "county_town_name")
df[categorical_cols] <- lapply(df[categorical_cols], as.character)  # Convert to character type
numerical_cols <- c("fips", "state", "county", "median2024", grep("s236|s235|bmir", names(df), value = TRUE))
df[numerical_cols] <- lapply(df[numerical_cols], as.numeric)

#3
df <- df %>% distinct()

#5
df <- df %>%
  mutate(geo_key = str_trim(paste(county_name, state_alpha, sep = ", ")))

#6
df_summary <- df %>%
  group_by(county_name) %>%
  summarise(
    median2024 = mean(median2024, na.rm = TRUE),
    s236_1 = mean(s236_1, na.rm = TRUE),
    s235_1 = mean(s235_1, na.rm = TRUE),
    bmir_1 = mean(bmir_1, na.rm = TRUE)
  )

#6 with state
df_state_summary <- df %>%
  group_by(state_alpha) %>%
  summarise(
    median2024 = mean(median2024, na.rm = TRUE),
    s236_1 = mean(s236_1, na.rm = TRUE),
    s235_1 = mean(s235_1, na.rm = TRUE),
    bmir_1 = mean(bmir_1, na.rm = TRUE)
  )

#7
df <- df %>%
  mutate(income_percentile = (median2024 - min(median2024, na.rm = TRUE)) /
                             (max(median2024, na.rm = TRUE) - min(median2024, na.rm = TRUE)))

#7
income_cols <- c("median2024", grep("s236|s235|bmir", names(df), value = TRUE))

df[income_cols] <- as.data.frame(lapply(df[income_cols], function(x) rescale(x, to = c(0, 1))))

#save clean dataset
write_csv(df, "/content/cleaned_section221.csv")
write_csv(df_summary, "/content/county_summary.csv")
write_csv(df_state_summary, "/content/state_summary.csv")



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘scales’


The following object is masked from ‘package:readr’:

    col_factor


Rows: 4764 Columns: 35
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): fips, State_Alpha, state, state_name, hud_area_code, hud_area_name...
dbl (26): metro, median2024, S236_1, S236_2, S236_3, S236_4, S236_5, S236_6,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
